In [3]:
import cv2
import os
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()
import preprocessor

In [4]:
def main():
    print('OpenCV version {} '.format(cv2.__version__))
    path='E:/7th Sem/Biometrics/Project/signature/data'
    current_dir = os.path.dirname(path)

    author = '029'
    training_folder = os.path.join(current_dir, 'data/training/', author)
    test_folder = os.path.join(current_dir, 'data/test/', author)

    training_data = []
    training_labels = []
    for filename in os.listdir(training_folder):
        img = cv2.imread(os.path.join(training_folder, filename), 0)
        if img is not None:
            data = preprocessor.prepare(img)
            training_data.append(data)
            training_labels.append([0, 1] if "genuine" in filename else [1, 0])

    test_data = []
    test_labels = []
    for filename in os.listdir(test_folder):
        img = cv2.imread(os.path.join(test_folder, filename), 0)
        if img is not None:
            data = preprocessor.prepare(img)
            test_data.append(data)
            test_labels.append([0, 1] if "genuine" in filename else [1, 0])

    sgd(training_data, training_labels, test_data, test_labels)


# Softmax Regression Model
def regression(x):
    W = tf.Variable(tf.zeros([901, 2]), name="W")
    b = tf.Variable(tf.zeros([2]), name="b")
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    return y, [W, b]


def sgd(training_data, training_labels, test_data, test_labels):
    # model
    with tf.compat.v1.variable_scope("regression"):
        x = tf.compat.v1.placeholder(tf.float32, [None, 901])
        y, variables = regression(x)

    # train
    y_ = tf.compat.v1.placeholder("float", [None, 2])
    cross_entropy = -tf.reduce_sum(y_ * tf.math.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(train_step, feed_dict={x: training_data, y_: training_labels})
        print(sess.run(accuracy, feed_dict={x: test_data, y_: test_labels}))


In [5]:
if __name__ == '__main__':
    main()

OpenCV version 4.6.0 
0.5


In [6]:
import numpy as np
TRAIN_DIR="E:/7th Sem/Biometrics/Project/signature/data/training/"
def get_images(fish):
 """Load files from train folder"""
 fish_dir = TRAIN_DIR+'{}'.format(fish)
 images = [fish+'/'+im for im in os.listdir(fish_dir)]
 return images
SIGNATURE_CLASSES = ['021','024','029']
files = []
y_all = []
for fish in SIGNATURE_CLASSES:
 fish_files = get_images(fish)
 files.extend(fish_files)
 y_fish = np.tile(fish, len(fish_files))
 y_all.extend(y_fish)
 print("{0} photos of {1}".format(len(fish_files), fish))
y_all = np.array(y_all)
print(len(files))
print(len(y_all))

24 photos of 021
24 photos of 024
23 photos of 029
71
71
